In [1]:
soap_notes_fp = "sample_output.csv"

In [2]:
print("soap_notes_fp =", soap_notes_fp)

soap_notes_fp = sample_output.csv


# SOAP Note Generation

This notebook performs step 1 of the Brief Hospital Course pipeline, in which we generate a SOAP note for each service based on a list of information that has been passed into a GPT-3.5 model. 

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

import os
import openai

In [5]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [6]:
from preprocessing import *
from prompt_functions import create_pt_prompt_per_day, create_pt_prompt_per_service

## Read in Prompts

In [7]:
prompts = pd.read_csv("soap_note_prompts.csv")

## Read in Radiology Reports

In [8]:
radiology = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/radiology.csv.gz")

## Read in Structured Data

In [9]:
###### 
# Challenge Data

# discharge summaries
discharges = pd.read_csv("/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/discharge.csv.gz")

# ed stays
edstays = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/edstays.csv.gz')

# triage
triage = pd.read_csv('/gpfs/milgram/project/rtaylor/shared/DischargeMe/public/train/triage.csv.gz')

###### 
# MIMIC-IV Data

# ward transfers
transfers = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_transfers.pkl')

# higher-level services (ICU, CARD, etc)
services = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_services.pkl')

# get patient info
pts = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_patients.pkl')

# admission demographics
admissions = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_admissions.pkl')

# procedures
procs = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_procedures_icd.pkl')

# diagnoses
diags = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_diagnoses_icd.pkl')

# meds
med_orders = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_emar.pkl')


In [10]:
prescriptions = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_prescriptions.pkl')
labs = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_labevents.pkl')
microbio = pd.read_pickle('/gpfs/milgram/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_microbiologyevents.pkl')

### Clean up/type cast data

In [11]:
transfers.loc['hadm_id'] = pd.to_numeric(transfers["hadm_id"], errors="ignore")

/tmp/ipykernel_4087343/85971565.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  transfers.loc['hadm_id'] = pd.to_numeric(transfers["hadm_id"], errors="ignore")


In [12]:
med_orders = med_orders[~med_orders['charttime'].isna() & 
                        ~med_orders['medication'].isna() & 
                        ~med_orders['event_txt'].isna()]

In [13]:
procs = procs.astype({"chartdate":"datetime64[ns]"})
med_orders = med_orders.astype({"charttime":"datetime64[ns]",
                  "scheduletime":"datetime64[ns]",
                  "storetime":"datetime64[ns]",})

discharges = discharges.astype({"charttime":"datetime64[ns]",
                               "storetime":"datetime64[ns]"})

In [26]:
edstays = edstays.astype({"intime":"datetime64[ns]",
                         "outtime":"datetime64[ns]"})

In [14]:
prescriptions = prescriptions.astype({"starttime":"datetime64[ns]",
                      "stoptime":"datetime64[ns]"})
labs = labs.astype({"charttime":"datetime64[ns]",
                      "storetime":"datetime64[ns]"})

microbio = microbio.astype({"chartdate":"datetime64[ns]",
                      "charttime":"datetime64[ns]",
                           "storetime":"datetime64[ns]",
                      "charttime":"datetime64[ns]"})

In [15]:
# generate free text sections of tables concatenating columns
med_orders['admin_text'] = med_orders['medication'] + " at " + med_orders['charttime'].dt.strftime('%B %d, %Y, %r')
# TODO SORAYA: Add the right information in here that we actually want for the meds
prescriptions['text'] = prescriptions['drug'] + " " + prescriptions['prod_strength']


In [48]:
# TODO THOMAS: figure out how to better encapsulate lab values
labs['text'] = (labs['category'].fillna("") + " " + labs['fluid'].fillna("") + " " 
                + labs['label'].fillna("") + ": " + labs['value'].fillna("") + " " 
                + labs['valueuom'].fillna(""))

In [50]:
# TODO SORAYA: check for those patients that don't have comments (they might not have useful information in the test)
microbio['text'] = microbio['test_name'].fillna("") + ": " + microbio['comments'].fillna("")

## Generate Input Data List Prompt

In [67]:
z = discharges.sample(1).squeeze()

In [72]:
service_prompts = create_pt_prompt_per_service(z, prompts, 
                                               edstays, 
                                               pts, triage, transfers, diags, procs, prescriptions, labs, microbio)

In [73]:
print(service_prompts)

["___ is a 57 year old F that initially presented to the ED with L Facial droop, Slurred speech, Transfer. By the end of the hospital stay, ___ was given the following diagnoses in order of importance to this admission: Disorder of brain, unspecified, Cerebral edema, Fasciculation, Facial weakness, Dysphasia, Noninflammatory disorder of uterus, unspecified, Melanoma in situ of other parts of face, Other localized visual field defect, left eye, Unspecified abnormalities of gait and mobility.A full list of the patient's procedures, medications, labs, and lab cultures is given below. Procedures are ordered by priority and medications are ordered chronologically. Given this information, please generate a SOAP progress note for this patient for their care during this part of their hospital course staying in the Emergency Department ward. Be SPECIFIC to the conditions, any abnormal labs, vitals, or procedures that were conducted, and significant medications as they relate to the hospital cou

## Create SOAP notes from GPT API

In [ ]:
import openai
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
engine = "decile-gpt-35-turbo-16k"


In [ ]:
# drop all eventtypes of type: discharge, since we don't need a note for that
service_prompts = service_prompts[service_prompts['eventtype'] != "discharge"]

In [ ]:
gpt_inputs = []
for _, row in service_prompts.iterrows():
    system_role = prompts['SOAP_system_message']
    system_role = system_role.replace(r"{{ward}}", row['careunit'])
    message_text = [{"role":"system","content":system_role}]
    
    gpt_service_prompt = {"role":"user",
                     "content":row['service_prompts']}
    
    message_text.append(gpt_service_prompt)
    
    # print(f"Deprescribe Prompt: {message_text}")
    gpt_inputs.append(message_text)

In [ ]:
completions = []
for input in gpt_inputs:
    completion = openai.ChatCompletion.create(
      engine=engine,
      messages = input,
    )
    completions.append(completion['choices'][0]['message']['content'])

In [ ]:
service_prompts['gpt_SOAP_note'] = completions

In [ ]:
service_prompts.to_csv(soap_notes_fp)